In [ ]:
#pip install numpy,minisom,scikit-learn,tqdm,SimpleITK,matplotlib

In [ ]:
import numpy as np
from minisom import MiniSom
from sklearn.metrics import silhouette_score

def som_silhouette(som, data):
    # Assign data points to clusters based on BMU
    labels = np.array([som.winner(x)[0] for x in data])

    # Calculate Silhouette Coefficient
    silhouette_avg = silhouette_score(data, labels)
    #print('avg_silhoutte',silhouette_avg)
    return silhouette_avg


In [ ]:
import minisom

In [ ]:
np_six_color=np.array([
    [0.764, 0.863, 0.933],  # Light blue (calming, airy)
    [0.294, 0.533, 0.851],  # Royal blue (trustworthy, professional)
    [0.988, 0.737, 0.831],  # Light pink (soft, romantic)
    [0.937, 0.831, 0.706],  # Light brown (natural, earthy)
    [0.439, 0.859, 0.392],  # Forest green (refreshing, serene)
    [0.996, 0.878, 0.631],  # Peach (warm, cheerful)
], dtype=np.float32)

In [ ]:
import tqdm
import os,time
from datetime import datetime
import numpy as np
from minisom import MiniSom
from sklearn.metrics import silhouette_score
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom
import pandas as pd

fts=[]
gb_avs=[]
pat=[]
#C://Users//Administrator//Desktop//gb_voxel_outputs//20_mm/TMH/gldm/120433028/
#Mainpath='C://Users//Administrator//Desktop//gb_voxel_outputs//20_mm/TMH/gldm/'
Mainpath='C://Users//Administrator//Desktop/GB_ANALYSIS/GB_ANALYSIS_new/20_mm_final/TMH/gldm/'
for gb in tqdm.tqdm(os.listdir(Mainpath)):
    pat.append(gb)
    fts1=[]
    gb_avs1=[]
    Pathnrrd=Mainpath+gb+"/"
    ##print(Pathnrrd)
    for dirName, subdirList, fileList in(os.walk(Pathnrrd)):
           # print(dirName)
            for filename in fileList:
                #print(filename)

                start_time = datetime.now()
                print(start_time)
                print("Starting operation...")
                if ".nrrd" in filename.lower():
                        print(time.time)
                        avs=[]
                        ##print('_____________________________________________________________________________________')
                        #c+=1
                        print(filename)
                        fts1.append(filename)
                        ##print('full file path\n',os.path.join(dirName, filename))
                        #ds = pydicom.filereader.dcmread(os.path.join(dirName, filename))
                        #image = sitk.ReadImage('C://Users//Administrator//Desktop//gb_voxel_outputs//20_mm/lung1/gldm/LUNG1-010/LUNG1-010_original_gldm_DependenceNonUniformity.nrrd')
                        image = sitk.ReadImage(os.path.join(dirName, filename))
                        image_array = sitk.GetArrayFromImage(image)



                        for i in range(1,image_array.shape[0]-1):
                            ##print('_____________________________________________________________________________________________________________')
                            ##print('shape:',image_array.shape)
                            # Check the image dimensions (assuming the image is in shape (depth, height, width, channels) or (depth, height, width))
                            if image_array.ndim not in (3, 4):
                                raise ValueError("Image array must be a 3D or 4D array")

                            # If the image has more than one channel, assume the last dimension is the channel dimension
                            if image_array.ndim == 3:
                                # Select a specific slice for 2D segmentation
                                slice_index = i # Adjust the slice index as needed
                                if slice_index >= image_array.shape[0]:
                                    raise ValueError(f"Invalid slice index. Maximum index allowed: {image_array.shape[0] - 1}")
                                image_slice = image_array[slice_index]
                            else:
                                # The image is already 3D, select a default slice
                                image_slice = image_array[image_array.shape[0] // 2]

                            # Normalize the image slice
                            image_slice = image_slice.astype(np.float32) / np.max(image_slice)

                            # Ensure the slice is 2D for display and processing
                            if image_slice.ndim == 3 and image_slice.shape[2] not in (1, 3):
                                # If it's neither single-channel nor RGB, pick the first channel
                                image_slice = image_slice[:, :, 0]

                            # Reshape the image to be a list of pixels
                            if image_slice.ndim == 3:
                                # For multi-channel images
                                pixels = image_slice.reshape(-1, image_slice.shape[2])
                                input_len = image_slice.shape[2]
                            else:
                                # For grayscale images
                                pixels = image_slice.reshape(-1, 1)
                                input_len = 1

                            # Define the SOM
                            som_width =2
                            som_height = 2
                            som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.8)
                            som.random_weights_init(pixels)

                            # Train the SOM
                            ##print("Training the SOM...")
                            ##print(i)
                            #som.train_random(pixels, 10000,verbose= True)
                            som.train(pixels, 1000,verbose= False)

                            st=som_silhouette(som=som, data=pixels)
                            avs.append(st)

                            # Map each pixel to the SOM and assign a label
                            ##print("Segmenting the image...")
                            labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)
                            ##print('Number of labels',len(labels))
                            for i in range(image_slice.shape[0]):
                                for j in range(image_slice.shape[1]):
                                    pixel = image_slice[i, j] if input_len > 1 else [image_slice[i, j]]
                                    w = som.winner(pixel)
                                    labels[i, j] = w[0] * som_width + w[1]

                            # Map the labels to colors
                            #label_colors = np.random.rand(som_width * som_height, 3)
                            label_colors = np_six_color
                            #label_colors=np_six_color
                            segmented_image = label_colors[labels]

                            # Rescale segmented image to [0, 255] for better display
                            segmented_image = (segmented_image * 255).astype(np.uint8)

    #                         # Plot the original and segmented images side by side
    #                         fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    #                         # Original image
    #                         if image_slice.ndim == 2:
    #                             axes[0].imshow(image_slice, cmap='bone')
    #                         else:
    #                             axes[0].imshow(image_slice)
    #                         axes[0].set_title(f'Input Image (Slice {slice_index})')
    #                         axes[0].axis('off')

    #                         #print('\t\t\t',re.split('/',nrrd_path)[-1])
    #                         print('\t\t\t',filename)
    #                         # Segmented image
    #                         axes[1].imshow(segmented_image)
    #                         #axes[1].set_title('Segmented Image',som_width,' X ',som_height, 'grid')
    #                         axes[1].set_title(f'Segmented Image ({som_width}x{som_height} grid)')

    #                         axes[1].axis('off')

    #                         plt.show()
                        
                        print('Average Silhoutte score:',np.sum(np.array(avs))/len(avs))
                        print('________________________________________________________')
                        gb_avs1.append(np.sum(np.array(avs))/len(avs))

    fts.append(fts1)
    gb_avs.append(gb_avs1)            
end_time = datetime.now()

# Calculate the duration
duration = end_time - start_time

print(duration)  


In [ ]:
167*14

In [ ]:
pwd

In [ ]:
import pandas as pd
final_silhoutte=pd.DataFrame(gb_avs,columns=[fts[0]],index=pat)
final_silhoutte.to_csv('TMH_average_silhoutte_score_new')
print('done saving to csv')
final_silhoutte

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

# Load the NRRD image using SimpleITK
#nrrd_path = 'C://Users//Administrator//Desktop//gb_voxel_outputs//20_mm/TMH/gldm/110348284/110348284_original_gldm_HighGrayLevelEmphasis.nrrd'  # Replace with your NRRD image path
nrrd_path ="C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_HighGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check the image dimensions (assuming the image is in shape (depth, height, width, channels) or (depth, height, width))
if image_array.ndim not in (3, 4):
    raise ValueError("Image array must be a 3D or 4D array")

# If the image has more than one channel, assume the last dimension is the channel dimension
if image_array.ndim == 3:
    # Ensure we have enough slices to plot
    num_slices = min(10, image_array.shape[0])
else:
    num_slices = 1

# Normalize the image slice
def normalize_slice(slice):
    return (slice.astype(np.float32) / np.max(slice)).clip(0, 1)

# Define the SOM parameters
som_width = 2
som_height =2
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

#label_colors = np.random.rand(som_width * som_height, 3)

# Create a figure with subplots for each slice
fig, axes = plt.subplots(10, 2, figsize=(12, 30))

for i in range(num_slices):
    # Select the slice
    image_slice = image_array[i]

    # Normalize the image slice
    image_slice_norm = normalize_slice(image_slice)

    # Reshape the image to be a list of pixels
    pixels = image_slice_norm.reshape(-1, input_len)

    # Initialize and train the SOM
    som.random_weights_init(pixels)
    som.train_random(pixels, 1000)

    # Map each pixel to the SOM and assign a label
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_width + w[1]

    # Map the labels to colors
    #label_colors = np.random.rand(som_width * som_height, 3,s)
    segmented_image = label_colors[labels]

    # Rescale segmented image to [0, 255] for better display
    segmented_image = (segmented_image * 255).astype(np.uint8)

    # Plot the original and segmented images
    axes[i, 0].imshow(image_slice_norm, cmap='gray' if input_len == 1 else None)
    axes[i, 0].set_title(f'Original Image (Slice {i})')
    axes[i, 0].axis('off')

    axes[i, 1].imshow(segmented_image)
    axes[i, 1].set_title(f'Segmented Image (Slice {i})')
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_HighGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check the image dimensions
if image_array.ndim not in (3, 4):
    raise ValueError("Image array must be a 3D or 4D array")

# Determine the number of slices
num_slices = min(10, image_array.shape[0]) if image_array.ndim == 3 else 1

# Normalize the image slice
def normalize_slice(slice):
    return (slice.astype(np.float32) / np.max(slice)).clip(0, 1)

# Define the SOM parameters
som_width = 2
som_height = 2
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Create a figure with subplots for each slice
fig, axes = plt.subplots(num_slices, 2, figsize=(12, 30))

for i in range(num_slices):
    # Select the slice
    image_slice = image_array[i]

    # Normalize the image slice
    image_slice_norm = normalize_slice(image_slice)

    # Reshape the image to be a list of pixels
    pixels = image_slice_norm.reshape(-1, input_len)

    # Train the SOM without random initialization
    som.train_random(pixels, 1000)

    # Map each pixel to the SOM and assign a label
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_width + w[1]

    # Map the labels to colors
    label_colors = np.random.rand(som_width * som_height, 3)  # Ensure to generate colors here
    segmented_image = label_colors[labels]

    # Rescale segmented image to [0, 255] for better display
    segmented_image = (segmented_image * 255).astype(np.uint8)

    # Plot the original and segmented images
    axes[i, 0].imshow(image_slice_norm, cmap='gray' if input_len == 1 else None)
    axes[i, 0].set_title(f'Original Image (Slice {i})')
    axes[i, 0].axis('off')

    axes[i, 1].imshow(segmented_image)
    axes[i, 1].set_title(f'Segmented Image (Slice {i})')
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_HighGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim not in (3, 4):
    raise ValueError("Image array must be a 3D or 4D array")

if image_array.ndim == 3:
    num_slices = min(10, image_array.shape[0])
else:
    num_slices = 1

def normalize_slice(slice):
    return (slice.astype(np.float32) / np.max(slice)).clip(0, 1)

# Define SOM parameters
som_width = 2
som_height = 2
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Create a DataFrame to hold features
features = []

for i in range(num_slices):
    # Select the slice
    image_slice = image_array[i]
    
    # Normalize the image slice
    image_slice_norm = normalize_slice(image_slice)
    
    # Reshape image to list of pixels
    pixels = image_slice_norm.reshape(-1, input_len)

    # Train the SOM
    som.random_weights_init(pixels)
    som.train_random(pixels, 1000)

    # Map each pixel to the SOM and assign a label
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_width + w[1]

    # Extract features from each cluster
    for cluster in range(som_width * som_height):
        cluster_pixels = pixels[labels.flatten() == cluster]
        
        if cluster_pixels.size > 0:
            mean_feature = np.mean(cluster_pixels, axis=0)
            std_feature = np.std(cluster_pixels, axis=0)
            features.append(np.concatenate([[i, cluster], mean_feature, std_feature]))  # i: slice index, cluster: cluster label

# Convert features to DataFrame
feature_array = np.array(features)
columns = ['Slice', 'Cluster'] + [f'Feature_{j}' for j in range(input_len)] + [f'Std_Feature_{j}' for j in range(input_len)]
features_df = pd.DataFrame(feature_array, columns=columns)

# Standardize features if necessary
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_df.iloc[:, 2:].astype(float))
scaled_features_df = pd.DataFrame(scaled_features, columns=features_df.columns[2:])

# Now, scaled_features_df can be used as input to the Cox model
print(scaled_features_df)

# Continue with the Cox model implementation using the features


In [ ]:
845+1075+575+1424+632+684


In [ ]:
scaled_features_df

In [ ]:
num_slices

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_HighGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim not in (3, 4):
    raise ValueError("Image array must be a 3D or 4D array")

if image_array.ndim == 3:
    num_slices = min(10, image_array.shape[0])
else:
    num_slices = 1

def normalize_slice(slice):
    return (slice.astype(np.float32) / np.max(slice)).clip(0, 1)

# Define SOM parameters
som_width = 2
som_height = 2
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Dictionary to hold pixels grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height)}

for i in range(num_slices):
    # Select the slice
    image_slice = image_array[i]
    
    # Normalize the image slice
    image_slice_norm = normalize_slice(image_slice)
    
    # Reshape image to list of pixels
    pixels = image_slice_norm.reshape(-1, input_len)

    # Train the SOM
    som.random_weights_init(pixels)
    som.train_random(pixels, 1000)

    # Map each pixel to the SOM and assign a label
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_width + w[1]

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]  # Use original pixel value (before normalization)
            pixel_clusters[cluster_label].append(pixel_value)

# Plot the pixel value distributions for each cluster
plt.figure(figsize=(12, 6))

for label, pixels in pixel_clusters.items():
    plt.hist(pixels, bins=30, alpha=0.6, label=f'Cluster {label}', density=True)

plt.title('Pixel Value Distribution by Cluster')
plt.xlabel('Pixel Value')
plt.ylabel('Density')
plt.legend()
plt.grid()
plt.show()


In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_SmallDependenceLowGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim not in (3, 4):
    raise ValueError("Image array must be a 3D or 4D array")

if image_array.ndim == 3:
    num_slices = min(10, image_array.shape[0])
else:
    num_slices = 1

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)  # Return an array of zeros if max is zero

# Define SOM parameters
som_width = 2
som_height = 2
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Dictionary to hold pixels grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height)}

for i in range(num_slices):
    # Select the slice
    image_slice = image_array[i]
    
    # Normalize the image slice
    image_slice_norm = normalize_slice(image_slice)
    
    # Reshape image to list of pixels
    pixels = image_slice_norm.reshape(-1, input_len)

    # Train the SOM
    som.random_weights_init(pixels)
    #som.train_random(pixels, 1000)
    som.train(pixels, 1000)

    # Map each pixel to the SOM and assign a label
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_width + w[1]

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]  # Use original pixel value (before normalization)
            pixel_clusters[cluster_label].append(pixel_value)

    # Visualize the clusters as images
    cluster_images = []
    for label in range(som_width * som_height):
        # Create an empty image for the cluster
        cluster_image = np.zeros_like(image_slice, dtype=np.float32)
        # Assign pixel values to the cluster image where labels match
        cluster_image[labels == label] = image_slice[labels == label]
        cluster_images.append(cluster_image)

    # Plot the original and cluster images
    fig, axes = plt.subplots(1, som_width * som_height + 1, figsize=(15, 5))
    
    # Plot the original image
    axes[0].imshow(image_slice_norm, cmap='gray')
    axes[0].set_title(f'Original Image (Slice {i})')
    axes[0].axis('off')

    # Plot each cluster image
    for j, cluster_image in enumerate(cluster_images):
        axes[j + 1].imshow(cluster_image, cmap='gray')
        axes[j + 1].set_title(f'Cluster {j}')
        axes[j + 1].axis('off')

    plt.tight_layout()
    plt.show()


### 3d SOM

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_HighGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim != 3:
    raise ValueError("Image array must be a 3D array")

num_slices = image_array.shape[0]

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)  # Return an array of zeros if max is zero

# Define 3D SOM parameters
som_width = 2
som_height = 2
som_depth = 1  # New dimension for 3D
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Dictionary to hold pixels grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height * som_depth)}

for i in range(num_slices):
    # Select the slice
    image_slice = image_array[i]
    
    # Normalize the image slice
    image_slice_norm = normalize_slice(image_slice)
    
    # Reshape image to list of pixels
    pixels = image_slice_norm.reshape(-1, input_len)

    # Train the SOM
    som.random_weights_init(pixels)
    som.train(pixels, 1000)

    # Map each pixel to the SOM and assign a label
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_height * som_depth + w[1] * som_depth + (i % som_depth)

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]  # Use original pixel value (before normalization)
            pixel_clusters[cluster_label].append(pixel_value)

    # Visualize the clusters as images
    cluster_images = []
    for label in range(som_width * som_height * som_depth):
        # Create an empty image for the cluster
        cluster_image = np.zeros_like(image_slice, dtype=np.float32)
        # Assign pixel values to the cluster image where labels match
        cluster_image[labels == label] = image_slice[labels == label]
        cluster_images.append(cluster_image)

    # Plot the original and cluster images
    num_clusters = som_width * som_height * som_depth
    fig, axes = plt.subplots(1, num_clusters + 1, figsize=(15, 5))

    # Plot the original image
    axes[0].imshow(image_slice_norm, cmap='gray')
    axes[0].set_title(f'Original Image (Slice {i})')
    axes[0].axis('off')

    # Plot each cluster image
    for j in range(num_clusters):
        if j < len(cluster_images):  # Check if cluster_image exists
            axes[j + 1].imshow(cluster_images[j], cmap='hot')
            axes[j + 1].set_title(f'Cluster {j}')
            axes[j + 1].axis('off')
        else:
            axes[j + 1].axis('off')  # Hide empty axes if no cluster image exists

    plt.tight_layout()
    plt.show()


In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_SmallDependenceLowGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim != 3:
    raise ValueError("Image array must be a 3D array")

num_slices = image_array.shape[0]

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)

# Define SOM parameters
som_width = 2
som_height = 2
som_depth = 1
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Prepare a list to hold pixel values from all slices
all_pixels = []

# Normalize and gather all pixel values from all slices
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    pixels = image_slice_norm.reshape(-1, input_len)
    all_pixels.append(pixels)

# Concatenate all pixel values into a single array
all_pixels = np.vstack(all_pixels)

# Train the SOM on the combined pixel values
som.random_weights_init(all_pixels)
som.train(all_pixels, 1000)

# Initialize a dictionary to hold pixel values grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height * som_depth)}

# Now map each pixel in each slice to the trained SOM
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)

    # Map each pixel to the SOM
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_height * som_depth + w[1] * som_depth + (i % som_depth)

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]
            pixel_clusters[cluster_label].append(pixel_value)

    # Visualize the clusters as images
    cluster_images = []
    for label in range(som_width * som_height * som_depth):
        cluster_image = np.zeros_like(image_slice, dtype=np.float32)
        cluster_image[labels == label] = image_slice[labels == label]
        cluster_images.append(cluster_image)

    # Plot the original and cluster images
    num_clusters = som_width * som_height * som_depth
    fig, axes = plt.subplots(1, num_clusters + 1, figsize=(15, 5))

    # Plot the original image
    axes[0].imshow(image_slice_norm, cmap='gray')
    axes[0].set_title(f'Original Image (Slice {i})')
    axes[0].axis('off')

    # Plot each cluster image
    for j in range(num_clusters):
        if j < len(cluster_images):  # Check if cluster_image exists
            axes[j + 1].imshow(cluster_images[j], cmap='gray')
            axes[j + 1].set_title(f'Cluster {j}')
            axes[j + 1].axis('off')
        else:
            axes[j + 1].axis('off')  # Hide empty axes if no cluster image exists

    plt.tight_layout()
    plt.show()


In [ ]:
pwd

### GB Yellamma finally working

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/108689672/108689672_original_gldm_GrayLevelVariance.nrrd"

import re 
r1=re.split(string=nrrd_path, pattern='\/')
r2=re.split(string=r1[-1], pattern='_')
fname=r2[-1]

image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim != 3:
    raise ValueError("Image array must be a 3D array")

num_slices = image_array.shape[0]

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)

# Define larger SOM parameters
som_width = 2
som_height= 2
som_depth = 1  # Increase depth for 3D SOM
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Prepare a list to hold pixel values from all slices
all_pixels = []

# Normalize and gather all pixel values from all slices
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    pixels = image_slice_norm.reshape(-1, input_len)
    all_pixels.append(pixels)

# Concatenate all pixel values into a single array
all_pixels = np.vstack(all_pixels)

# Train the SOM on the combined pixel values
som.random_weights_init(all_pixels)
som.train(all_pixels, 10000)  # Increased training iterations

# Initialize a dictionary to hold pixel values grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height * som_depth)}

# Map each pixel in each slice to the trained SOM
#for i in range(1, num_slices - 1):
for i in range(1, num_slices -1):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)

    # Map each pixel to the SOM
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_height * som_depth + w[1] * som_depth + (i % som_depth)

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]
            pixel_clusters[cluster_label].append(pixel_value)

    # Visualize the clusters as images
    cluster_images = []
    for label in range(som_width * som_height * som_depth):
        cluster_image = np.zeros_like(image_slice, dtype=np.float32)
        cluster_image[labels == label] = image_slice[labels == label]
        cluster_images.append(cluster_image)

    # Plot the original and cluster images
    num_clusters = som_width * som_height * som_depth
    fig, axes = plt.subplots(1, num_clusters + 1, figsize=(15, 5))

    # Plot the original image
    axes[0].imshow(image_slice_norm, cmap='gray')
    #axes[0].set_title(fname,f'(Slice {i})')
    axes[0].set_title(f'{fname} (Slice {i})')

    axes[0].axis('off')

    # Plot each cluster image
    for j in range(num_clusters):
        if j < len(cluster_images):  # Check if cluster_image exists
            im=axes[j + 1].imshow(cluster_images[j], cmap='hot')
            axes[j + 1].set_title(f'Cluster {j}')
            axes[j + 1].axis('off')
        
            # Add a colorbar for the current cluster image
            plt.colorbar(im, ax=axes[j + 1])
        
        else:
            axes[j + 1].axis('off')  # Hide empty axes if no cluster image exists

    plt.tight_layout()
    plt.show()


In [ ]:
# test on 5 Patients
base_path = 'C:/Users/Administrator/Desktop/GB_ANALYSIS/GB_ANALYSIS_new/20_mm_final/gb_test_5/'

In [ ]:
import os
import glob
base_path = 'C:/Users/Administrator/Desktop/GB_ANALYSIS/GB_ANALYSIS_new/20_mm_final/gb_test_5/'

def get_first_nrrd_in_folders(base_path):
    # Loop through each folder in the base path
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        
        # Check if it is a directory
        if os.path.isdir(folder_path):
            # Use glob to find .nrrd files in the current folder
            nrrd_files = glob.glob(os.path.join(folder_path, "*.nrrd"))
            
            # If we have any .nrrd files, return the first one
            if nrrd_files:
                print(f"First .nrrd file in folder {folder}: {nrrd_files[0]}")
            else:
                print(f"No .nrrd files found in folder {folder}")

# Usage example
#base_path = '/path/to/your/directory'
get_first_nrrd_in_folders(base_path)


In [ ]:
del(base_path)

In [ ]:
# get volumes from each clusters

In [ ]:
del(main_cluster)

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom
import re 
import tqdm

gb_main=[]

gb_pat=[]
ac1=[]
ac2=[]
ac3=[]
ac4=[]

# Load the NRRD image using SimpleITK
#nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/108689672/108689672_original_gldm_GrayLevelVariance.nrrd"
#base_path="C:/Users/Administrator/Desktop/GB_ANALYSIS/GB_ANALYSIS_new/20_mm_final/gb_test_5/"
base_path='C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/'
# Loop through each folder in the base path
for folder in  tqdm.tqdm(os.listdir(base_path)):
    folder_path = os.path.join(base_path, folder)
    
    # Check if it is a directory
    if os.path.isdir(folder_path):
        # Use glob to find .nrrd files in the current folder
        nrrd_files = glob.glob(os.path.join(folder_path, "*.nrrd"))

        # If we have any .nrrd files, return the first one
        if nrrd_files:
            #print(f"First .nrrd file in folder {folder}: {nrrd_files[0]}")

            r1=re.split(string=nrrd_files[6], pattern='\/')
            #print(r1[-1])
            r2=re.split(string=r1[-1], pattern='\\\\')
            #r2=re.split(string=r1[-1], pattern='_')
            #print(r2[-1])
            fname=r2[-1]
            r3=re.split(string=fname, pattern='_')[0]
            
            
            print(fname)
            test.append(fname)
            gb_pat.append(r3)
            
            print('reading image')
            image = sitk.ReadImage(nrrd_files[6])
            image_array = sitk.GetArrayFromImage(image)
            print('read image')

            # Check image dimensions
            if image_array.ndim != 3:
                raise ValueError("Image array must be a 3D array")
            num_slices = image_array.shape[0]
            print('number of slices = ',num_slices)

            
            def normalize_slice(slice):
                max_val = np.max(slice)
                if max_val > 0:
                    return (slice.astype(np.float32) / max_val).clip(0, 1)
                else:
                    return np.zeros_like(slice, dtype=np.float32)

                
            # Define larger SOM parameters
            som_width = 2
            som_height= 2
            som_depth = 1  # Increase depth for 3D SOM
            input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
            som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

            # Prepare a list to hold pixel values from all slices
            all_pixels = []

            # Normalize and gather all pixel values from all slices
            for i in range(num_slices):
                image_slice = image_array[i]
                image_slice_norm = normalize_slice(image_slice)
                pixels = image_slice_norm.reshape(-1, input_len)
                all_pixels.append(pixels)

            # Concatenate all pixel values into a single array
            all_pixels = np.vstack(all_pixels)

            # Train the SOM on the combined pixel values
            som.random_weights_init(all_pixels)
            print('training som')
            som.train(all_pixels, 30000)  # Increased training iterations

            # Initialize a dictionary to hold pixel values grouped by cluster labels
            pixel_clusters = {label: [] for label in range(som_width * som_height * som_depth)}

            # four clusters

            c1=[]
            c2=[]
            c3=[]
            c4=[]

            # Map each pixel in each slice to the trained SOM
            #for i in range(1, num_slices - 1):
            print('mapping voxels to trained SOM')
            #for i in range(1, num_slices -1):
            for i in range (num_slices):
                image_slice = image_array[i]
                image_slice_norm = normalize_slice(image_slice)
                labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)

                # Map each pixel to the SOM
                for x in range(image_slice.shape[0]):
                    for y in range(image_slice.shape[1]):
                        w = som.winner(image_slice_norm[x, y])
                        labels[x, y] = w[0] * som_height * som_depth + w[1] * som_depth + (i % som_depth)

                # Extract pixel values into lists grouped by cluster label
                for x in range(image_slice.shape[0]):
                    for y in range(image_slice.shape[1]):
                        cluster_label = labels[x, y]
                        pixel_value = image_slice[x, y]
                        pixel_clusters[cluster_label].append(pixel_value)

                # Visualize the clusters as images
                cluster_images = []
                for label in range(som_width * som_height * som_depth):
                    cluster_image = np.zeros_like(image_slice, dtype=np.float32)
                    cluster_image[labels == label] = image_slice[labels == label]
                    cluster_images.append(cluster_image)
                #print('creating 3d clusters')
                # join all the 2d clusters into 1 3d volume.
                c1.append(cluster_images[0]) # first cluster
                c2.append(cluster_images[1]) # second cluster
                c3.append(cluster_images[2]) # third cluster
                c4.append(cluster_images[3]) # fourth cluster

                # Plot the original and cluster images
                num_clusters = som_width * som_height * som_depth
                fig, axes = plt.subplots(1, num_clusters + 1, figsize=(15, 5))

                # Plot the original image
                axes[0].imshow(image_slice_norm, cmap='gray')
                #axes[0].set_title(fname,f'(Slice {i})')
                axes[0].set_title(f'{fname} (Slice {i})')

                axes[0].axis('off')

                # Plot each cluster image
                for j in range(num_clusters):
                    if j < len(cluster_images):  # Check if cluster_image exists
                        im=axes[j + 1].imshow(cluster_images[j], cmap='hot')
                        axes[j + 1].set_title(f'Cluster {j+1}')
                        axes[j + 1].axis('off')

                        # Add a colorbar for the current cluster image
                        plt.colorbar(im, ax=axes[j + 1])

                    else:
                        axes[j + 1].axis('off')  # Hide empty axes if no cluster image exists

                plt.tight_layout()
                plt.show()
            #print(c1)
            # applying PCA for each cluster 
            ac1.append(gb_pca(c1))
            ac2.append(gb_pca(c2))
            ac3.append(gb_pca(c3))
            ac4.append(gb_pca(c4))
            
main_cluster=[gb_pat,ac1,ac2,ac3,ac4]                         

final_pca_df=pd.DataFrame(main_cluster,index=['pat_id','cluster1','cluster2','cluster3','cluster4']).T

final_pca_df.to_csv(fname+'_voxrad_pca_on_4_SOM_clusters.csv')
            

In [ ]:
final_pca_df

In [ ]:
fname

In [ ]:
re.split(string=fname, pattern='_')[0]

In [ ]:
pd.DataFrame(main_cluster,index=['cluster1','cluster2','cluster3','cluster4']).T.to_csv('4clusters.csv')

In [ ]:
pd.DataFrame(main_cluster,index=['cluster1','cluster2','cluster3','cluster4']).T

In [ ]:
pwd

In [ ]:
ac1

In [ ]:
ac2

In [ ]:
ac3

In [ ]:
ac4

In [ ]:
len(c4)

In [ ]:
np.shape(np.array(c1))

In [ ]:
77*59*63

In [ ]:
77*3717

In [ ]:
n2.shape[0]

In [ ]:
s


In [ ]:
gb_pca(c1)

In [ ]:
gb_pca(c5)

In [ ]:
        def gb_pca(s):
            nc1=np.array(s)
            print('cluster  shape:\t',np.shape(nc1))
            #plt.imshow(n2.reshape(n2.shape[0],-1))
            ncf1=nc1.flatten()
            pca = PCA(n_components=1)
            X_pca = pca.fit_transform(ncf1.reshape(-1, 1))

            print('flattened shape:',X_pca.shape)
            X_pca[0]
            print('pc1 value:     ',float(X_pca[0]))
            return float(X_pca[0])

In [ ]:
n2=np.array(c2)
#plt.imshow(n2.reshape(n2.shape[0],-1))
n3=n2.flatten()

In [ ]:
n3.shape

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Apply PCA and reduce to 1 component
pca = PCA(n_components=1)
X_pca = pca.fit_transform(n3.reshape(-1, 1))

print(X_pca.shape)  

In [ ]:
X_pca

In [ ]:
float(X_pca[0])

In [ ]:
n2.shape

In [ ]:
# flattened PCA
#PCA on flattened data
import numpy as np
from sklearn.decomposition import PCA
from lifelines import CoxPHFitter
import pandas as pd

# Example data: 
# Let’s assume you have a list of volumetric data, where each volumetric feature is a 3D array
# data_vol1 (100 observations, X1 x Y1 x Z1), data_vol2 (100 observations, X2 x Y2 x Z2)

# Function to flatten the volumetric data (reshape 3D to 1D)
def flatten_volumetric_data(vol_data):
    return vol_data.reshape(vol_data.shape[0], -1)  # Flatten the 3D data to 2D (n_samples, n_features)

# Example data (Random generation for illustration)
# Let's assume the data has a shape (100, 10, 10, 10) for both volumetric features
data_vol1 = np.random.rand(167, 10, 10, 10)
data_vol2 = np.random.rand(167, 10, 10, 10)

# Flatten both volumetric data
data_vol1_flat = flatten_volumetric_data(data_vol1)
data_vol2_flat = flatten_volumetric_data(data_vol2)

# Apply PCA to each flattened volumetric feature
def apply_pca_and_get_first_component(data):
    pca = PCA(n_components=1)  # Reduce to 1 component (first principal component)
    pca_result = pca.fit_transform(data)
    print('pca_result',pca_result)
    print(len(pca_result))
    return pca_result

# Get PCA results (single component) for both features
pca_vol1 = apply_pca_and_get_first_component(data_vol1_flat)
pca_vol2 = apply_pca_and_get_first_component(data_vol2_flat)

# Create a DataFrame for Cox model input, assuming you have survival data (e.g., duration and event)
# Example: duration and event columns (randomly generated for illustration)
duration = np.random.rand(167) * 10  # Survival duration
event = np.random.randint(0, 2, 167)  # Event occurred (1) or censored (0)

# Combine the PCA results into a DataFrame (columns: PCA of vol1, PCA of vol2)
X_pca = np.hstack([pca_vol1, pca_vol2])
df = pd.DataFrame(X_pca, columns=['PCA_vol1', 'PCA_vol2'])
df['duration'] = duration
df['event'] = event

# Fit the Cox Proportional Hazards model using the lifelines package
cox = CoxPHFitter()
cox.fit(df, duration_col='duration', event_col='event')

# Print the Cox model summary
cox.print_summary()


In [ ]:
data_vol1_flat.shape

In [ ]:
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim != 3:
    raise ValueError("Image array must be a 3D array")

num_slices = image_array.shape[0]

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)

# Define larger SOM parameters
som_width = 2
som_height= 2
som_depth = 1  # Increase depth for 3D SOM
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Prepare a list to hold pixel values from all slices
all_pixels = []

# Normalize and gather all pixel values from all slices
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    pixels = image_slice_norm.reshape(-1, input_len)
    all_pixels.append(pixels)

# Concatenate all pixel values into a single array
all_pixels = np.vstack(all_pixels)

# Train the SOM on the combined pixel values
som.random_weights_init(all_pixels)
som.train(all_pixels, 10000)  # Increased training iterations

# Initialize a dictionary to hold pixel values grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height * som_depth)}


# four clusters

c1=[]
c2=[]
c3=[]
c4=[]

# Map each pixel in each slice to the trained SOM
#for i in range(1, num_slices - 1):
for i in range(1, num_slices -1):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)

    # Map each pixel to the SOM
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_height * som_depth + w[1] * som_depth + (i % som_depth)

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]
            pixel_clusters[cluster_label].append(pixel_value)

    # Visualize the clusters as images
    cluster_images = []
    for label in range(som_width * som_height * som_depth):
        cluster_image = np.zeros_like(image_slice, dtype=np.float32)
        cluster_image[labels == label] = image_slice[labels == label]
        cluster_images.append(cluster_image)
    
    # join all the 2d clusters into 1 3d volume.
    c1.append(cluster_images[0]) # first cluster
    c2.append(cluster_images[1]) # second cluster
    c3.append(cluster_images[2]) # third cluster
    c4.append(cluster_images[3]) # fourth cluster

    # Plot the original and cluster images
    num_clusters = som_width * som_height * som_depth
    fig, axes = plt.subplots(1, num_clusters + 1, figsize=(15, 5))

    # Plot the original image
    axes[0].imshow(image_slice_norm, cmap='gray')
    #axes[0].set_title(fname,f'(Slice {i})')
    axes[0].set_title(f'{fname} (Slice {i})')

    axes[0].axis('off')

    # Plot each cluster image
    for j in range(num_clusters):
        if j < len(cluster_images):  # Check if cluster_image exists
            im=axes[j + 1].imshow(cluster_images[j], cmap='hot')
            axes[j + 1].set_title(f'Cluster {j}')
            axes[j + 1].axis('off')
        
            # Add a colorbar for the current cluster image
            plt.colorbar(im, ax=axes[j + 1])
        
        else:
            axes[j + 1].axis('off')  # Hide empty axes if no cluster image exists

    plt.tight_layout()
    plt.show()

In [ ]:
plt.imshow(c1[0])


In [ ]:
np.shape(c1)

In [ ]:
np.shape(c2)

In [ ]:
47*50*44

In [ ]:
all_pixels.shape

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/108689672/108689672_original_gldm_GrayLevelVariance.nrrd"

import re 
r1 = re.split(string=nrrd_path, pattern='\/')
r2 = re.split(string=r1[-1], pattern='_')
fname = r2[-1]

image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim != 3:
    raise ValueError("Image array must be a 3D array")

num_slices = image_array.shape[0]

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)

# Define SOM parameters (2x2 grid with 4 clusters)
som_width = 2
som_height = 2
som_depth = 1  # This is a 2D SOM for simplicity, can extend to 3D if needed
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Prepare a list to hold pixel values from all slices
all_pixels = []

# Normalize and gather all pixel values from all slices
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    pixels = image_slice_norm.reshape(-1, input_len)
    all_pixels.append(pixels)

# Concatenate all pixel values into a single array
all_pixels = np.vstack(all_pixels)

# Train the SOM on the combined pixel values
som.random_weights_init(all_pixels)
som.train(all_pixels, 10000)  # Increased training iterations

# Initialize a dictionary to hold 3D volumes for each cluster (4 clusters total)
cluster_3d_volumes = {label: np.zeros_like(image_array[0], dtype=np.float32) for label in range(som_width * som_height)}

# Map each pixel in each slice to the trained SOM and fill in the 3D cluster volumes
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)

    # Map each pixel to the SOM
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_height + w[1]  # Simple label assignment for a 2x2 SOM grid

    # Assign pixel values to the corresponding cluster's 3D volume
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]
            cluster_3d_volumes[cluster_label][i, x, y] = pixel_value

# Now, visualize the 3D volumes for each cluster
num_clusters = som_width * som_height
for cluster_label in range(num_clusters):
    # Plot the 3D volume for the current cluster (slice by slice)
    fig, axes = plt.subplots(1, num_slices, figsize=(15, 5))

    # Loop through each slice for the current cluster
    for i in range(num_slices):
        axes[i].imshow(cluster_3d_volumes[cluster_label][i], cmap='hot')
        axes[i].set_title(f"Cluster {cluster_label} - Slice {i}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
cluster_3d_images[4].shape

In [ ]:
len(cluster_images)

In [ ]:
cluster_images[0].shape

In [ ]:
cluster_images[0].shape

In [ ]:
np.shape(cluster_images)


In [ ]:
np.shape(cluster_images)

In [ ]:
np.shape(cluster_images)

In [ ]:
plt.imshow(cluster_images[4])

###
### Apply PCA for Each Cluster and Extract the First Principal Component (PC1)
#### We’ll now apply PCA separately for each cluster and extract the first principal component (PC1), which will serve as the ### summary feature for each cluster.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Initialize an empty list to store the first principal component for each cluster
pca_features = []

# Standardize and apply PCA for each cluster
for cluster_idx in range(num_clusters):
    # Get the voxel data for the current cluster
    cluster_data = df.iloc[:, cluster_idx * voxels_per_cluster:(cluster_idx + 1) * voxels_per_cluster]
    
    # Standardize the data (important for PCA)
    scaler = StandardScaler()
    cluster_data_scaled = scaler.fit_transform(cluster_data)
    
    # Apply PCA and get the first principal component (PC1)
    pca = PCA(n_components=1)
    pca_result = pca.fit_transform(cluster_data_scaled)
    
    # Append the first principal component for this cluster to the list
    pca_features.append(pca_result)

# Convert the list of PCA features into a DataFrame
pca_features = np.hstack(pca_features)  # Concatenate the PC1 scores for all clusters
df_pca = pd.DataFrame(pca_features, columns=[f'PC_Cluster_{i+1}' for i in range(num_clusters)])

# Add survival data (time and event) back to the DataFrame
df_pca['time'] = df['time']
df_pca['event'] = df['event']


In [ ]:
for x,c in enumerate(cluster_images):
    print(x,"\n",c)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# Initialize an empty list to store the first principal component for each cluster
pca_features = []

# Standardize and apply PCA for each cluster
for c in cluster_images:
    # Get the voxel data for the current cluster
    #cluster_data = df.iloc[:, cluster_idx * voxels_per_cluster:(cluster_idx + 1) * voxels_per_cluster]
    cluster_data=c
    print('raw data', cluster_data)
    plt.imshow(cluster_data)
    
    # Standardize the data (important for PCA)
    scaler = StandardScaler()
    cluster_data_scaled = scaler.fit_transform(cluster_data)
    print('scaled data', cluster_data_scaled)
    plt.imshow(cluster_data_scaled)
    
    # Apply PCA and get the first principal component (PC1)
    pca = PCA(n_components=1)
    pca_result = pca.fit_transform(cluster_data_scaled)
    
    # Append the first principal component for this cluster to the list
    pca_features.append(pca_result)

# Convert the list of PCA features into a DataFrame
pca_features = np.hstack(pca_features)  # Concatenate the PC1 scores for all clusters
df_pca = pd.DataFrame(pca_features, columns=[f'PC_Cluster_{i+1}' for i in range(num_clusters)])

# # Add survival data (time and event) back to the DataFrame
# df_pca['time'] = df['time']
# df_pca['event'] = df['event']


### Scaled pca

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Initialize an empty list to store the first principal component for each cluster
pca_features = []

# Iterate through the clusters
for i, c in enumerate(cluster_images):
    # Get the voxel data for the current cluster (c is assumed to be a 2D array)
    cluster_data = c
    print(f'Raw data for cluster {i}:', cluster_data)
    
    # Plot raw data
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title(f'Raw Data - Cluster {i}')
    plt.imshow(cluster_data, cmap='hot')
    plt.colorbar()
    
    # Standardize the data (important for PCA)
    scaler = StandardScaler()
    cluster_data_scaled = scaler.fit_transform(cluster_data)
    print(f'Scaled data for cluster {i}:', cluster_data_scaled)
    
    # Plot scaled data
    plt.subplot(1, 2, 2)
    plt.title(f'Scaled Data - Cluster {i}')
    plt.imshow(cluster_data_scaled, cmap='hot')
    plt.colorbar()
    
    plt.show()  # Show the plots for raw and scaled data

    # Apply PCA and get the first principal component (PC1)
    pca = PCA(n_components=1)
    pca_result = pca.fit_transform(cluster_data_scaled)
    
    # Append the first principal component for this cluster to the list
    pca_features.append(pca_result)

    # Optionally, you can visualize the first principal component if needed
    plt.figure()
    plt.title(f'First Principal Component - Cluster {i}')
    plt.plot(pca_result)
    plt.xlabel('Sample Index')
    plt.ylabel('Principal Component Value')
    plt.show()


In [ ]:
np.shape(pca_features)

In [ ]:
pca_features[0]

In [ ]:
pca_result

In [ ]:
np.mean(pca_result)

In [ ]:
np.shape(c1)

### PCA on flattend data.

In [ ]:
#PCA on flattened data
import numpy as np
from sklearn.decomposition import PCA
from lifelines import CoxPHFitter
import pandas as pd

# Example data: 
# Let’s assume you have a list of volumetric data, where each volumetric feature is a 3D array
# data_vol1 (100 observations, X1 x Y1 x Z1), data_vol2 (100 observations, X2 x Y2 x Z2)

# Function to flatten the volumetric data (reshape 3D to 1D)
def flatten_volumetric_data(vol_data):
    return vol_data.reshape(vol_data.shape[0], -1)  # Flatten the 3D data to 2D (n_samples, n_features)

# Example data (Random generation for illustration)
# Let's assume the data has a shape (100, 10, 10, 10) for both volumetric features
data_vol1 = np.random.rand(100, 10, 10, 10)
data_vol2 = np.random.rand(100, 10, 10, 10)

# Flatten both volumetric data
data_vol1_flat = flatten_volumetric_data(data_vol1)
data_vol2_flat = flatten_volumetric_data(data_vol2)

# Apply PCA to each flattened volumetric feature
def apply_pca_and_get_first_component(data):
    pca = PCA(n_components=1)  # Reduce to 1 component (first principal component)
    pca_result = pca.fit_transform(data)
    return pca_result

# Get PCA results (single component) for both features
pca_vol1 = apply_pca_and_get_first_component(data_vol1_flat)
pca_vol2 = apply_pca_and_get_first_component(data_vol2_flat)

# Create a DataFrame for Cox model input, assuming you have survival data (e.g., duration and event)
# Example: duration and event columns (randomly generated for illustration)
duration = np.random.rand(100) * 10  # Survival duration
event = np.random.randint(0, 2, 100)  # Event occurred (1) or censored (0)

# Combine the PCA results into a DataFrame (columns: PCA of vol1, PCA of vol2)
X_pca = np.hstack([pca_vol1, pca_vol2])
df = pd.DataFrame(X_pca, columns=['PCA_vol1', 'PCA_vol2'])
df['duration'] = duration
df['event'] = event

# Fit the Cox Proportional Hazards model using the lifelines package
cox = CoxPHFitter()
cox.fit(df, duration_col='duration', event_col='event')

# Print the Cox model summary
cox.print_summary()


In [ ]:
# 2. Coefficients and Hazard Ratios:
# For PCA_vol1:

# coef: -0.13 — This is the estimated coefficient for PCA_vol1 in the Cox model. 
A negative coefficient suggests that as the value of PCA_vol1 increases, the hazard (risk of the event happening) decreases.
# In other words, higher values of PCA_vol1 are associated with a lower risk of the event.

# exp(coef): 0.88 — This is the hazard ratio (HR) corresponding to PCA_vol1.
# A hazard ratio of 0.88 means that for each unit increase in PCA_vol1,the hazard of the event decreases by 12%(since 1− 0.88=
# 0.12
# 1−0.88=0.12).
# se(coef): 0.12 — The standard error of the coefficient. This measures the variability or uncertainty in the estimated coefficient.
# 95% Confidence Interval: The 95% confidence interval for the coefficient is from -0.37 to 0.10. Since zero is included in this interval, we cannot conclusively say that the effect of PCA_vol1 is statistically significant.
# exp(coef) lower 95% / upper 95%: The confidence interval for the hazard ratio is from 0.69 to 1.11, indicating that PCA_vol1 could potentially have either a protective effect (HR < 1) or no effect (HR ≈ 1), and we cannot be sure of its direction of impact.

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

# Stratify by the median of PCA_vol1 for simplicity
df['group_vol1'] = (df['PCA_vol1'] > df['PCA_vol1'].median()).astype(int)

# Fit Kaplan-Meier estimator
kmf = KaplanMeierFitter()
kmf.fit(df['duration'], event_observed=df['event'], label="All Data")
kmf.plot_survival_function()

# Stratify by PCA_vol1 (grouping based on median value)
kmf.fit(df['duration'][df['group_vol1'] == 0], event_observed=df['event'][df['group_vol1'] == 0], label="Low PCA_vol1")
ax = kmf.plot_survival_function()

kmf.fit(df['duration'][df['group_vol1'] == 1], event_observed=df['event'][df['group_vol1'] == 1], label="High PCA_vol1")
kmf.plot_survival_function(ax=ax)

plt.title('Kaplan-Meier Survival Curves Stratified by PCA_vol1')
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

# Assuming 'df' is your dataframe with the required columns:
# 'duration' (time), 'event' (event indicator), 'PCA_vol1', 'PCA_vol2'

# Create a KaplanMeierFitter instance
kmf = KaplanMeierFitter()

# Stratify by the median of PCA_vol1
median_vol1 = df['PCA_vol1'].median()
df['group_vol1'] = (df['PCA_vol1'] > median_vol1).astype(int)

# Stratify by the median of PCA_vol2
median_vol2 = df['PCA_vol2'].median()
df['group_vol2'] = (df['PCA_vol2'] > median_vol2).astype(int)

# Plot Kaplan-Meier Curves for PCA_vol1 groups
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)  # Left plot for PCA_vol1
kmf.fit(df['duration'], event_observed=df['event'], label="All Data")
kmf.plot_survival_function()

kmf.fit(df['duration'][df['group_vol1'] == 0], event_observed=df['event'][df['group_vol1'] == 0], label="Low PCA_vol1")
ax = kmf.plot_survival_function()

kmf.fit(df['duration'][df['group_vol1'] == 1], event_observed=df['event'][df['group_vol1'] == 1], label="High PCA_vol1")
kmf.plot_survival_function(ax=ax)

plt.title('Kaplan-Meier Survival Curves for PCA_vol1')
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.legend()

# Plot Kaplan-Meier Curves for PCA_vol2 groups
plt.subplot(1, 2, 2)  # Right plot for PCA_vol2
kmf.fit(df['duration'], event_observed=df['event'], label="All Data")
kmf.plot_survival_function()

kmf.fit(df['duration'][df['group_vol2'] == 0], event_observed=df['event'][df['group_vol2'] == 0], label="Low PCA_vol2")
ax = kmf.plot_survival_function()

kmf.fit(df['duration'][df['group_vol2'] == 1], event_observed=df['event'][df['group_vol2'] == 1], label="High PCA_vol2")
kmf.plot_survival_function(ax=ax)

plt.title('Kaplan-Meier Survival Curves for PCA_vol2')
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.legend()

plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

# Assuming 'df' is your dataframe with the required columns:
# 'duration' (time), 'event' (event indicator), 'PCA_vol1', 'PCA_vol2'

# Create a KaplanMeierFitter instance
kmf = KaplanMeierFitter()

# Plot Kaplan-Meier Curves for PCA_vol1
plt.figure(figsize=(12, 6))

# Plot for all data
plt.subplot(1, 2, 1)
kmf.fit(df['duration'], event_observed=df['event'], label="All Data")
kmf.plot_survival_function()
plt.title('Kaplan-Meier Survival Curve for PCA_vol1')
plt.xlabel('Time')
plt.ylabel('Survival Probability')

# Plot Kaplan-Meier Curves for PCA_vol2
plt.subplot(1, 2, 2)
kmf.fit(df['duration'], event_observed=df['event'], label="All Data")
kmf.plot_survival_function()
plt.title('Kaplan-Meier Survival Curve for PCA_vol2')
plt.xlabel('Time')
plt.ylabel('Survival Probability')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

# Assuming 'df' is your dataframe with the required columns:
# 'duration' (time), 'event' (event indicator), 'PCA_vol1', 'PCA_vol2'

# Create a KaplanMeierFitter instance
kmf = KaplanMeierFitter()

# Create two new groups based on a threshold (e.g., median)
# This splits data into two groups: below median and above median
median_vol1 = df['PCA_vol1'].median()
median_vol2 = df['PCA_vol2'].median()

# Group data by whether PCA_vol1 and PCA_vol2 are above or below their respective medians
df['group_vol1'] = (df['PCA_vol1'] > median_vol1).astype(int)  # 1 for above median, 0 for below median
df['group_vol2'] = (df['PCA_vol2'] > median_vol2).astype(int)  # 1 for above median, 0 for below median

# Plot Kaplan-Meier Curves for both PCA_vol1 and PCA_vol2 on the same plot
plt.figure(figsize=(10, 6))

# Kaplan-Meier for PCA_vol1 (grouping based on median)
kmf.fit(df['duration'][df['group_vol1'] == 0], event_observed=df['event'][df['group_vol1'] == 0], label="Low PCA_vol1")
kmf.plot_survival_function()

kmf.fit(df['duration'][df['group_vol1'] == 1], event_observed=df['event'][df['group_vol1'] == 1], label="High PCA_vol1")
kmf.plot_survival_function()

# Kaplan-Meier for PCA_vol2 (grouping based on median)
kmf.fit(df['duration'][df['group_vol2'] == 0], event_observed=df['event'][df['group_vol2'] == 0], label="Low PCA_vol2")
kmf.plot_survival_function()

kmf.fit(df['duration'][df['group_vol2'] == 1], event_observed=df['event'][df['group_vol2'] == 1], label="High PCA_vol2")
kmf.plot_survival_function()

# Customize the plot
plt.title('Kaplan-Meier Survival Curves for PCA_vol1 and PCA_vol2')
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.legend(title="Groups", loc='best')
plt.tight_layout()
plt.show()


In [ ]:
df

In [ ]:
# Key Plots:
# Kaplan-Meier Curves: To visualize survival by feature groups.
# Hazard Function Plot: To see the predicted hazard over time for different feature values.
# Cumulative Hazard Plot: To see the cumulative hazard over time for different feature values.
# Schoenfeld Residuals: To check the proportional hazards assumption.
# Forest Plot: To visualize the hazard ratios and their confidence intervals.
# Martingale Residuals: To assess the goodness of fit of the Cox model.

In [ ]:
df['PCA_vol1'].min()

In [ ]:
 np.linspace(df['PCA_vol1'].min(), df['PCA_vol1'].max(), 100)

In [ ]:
# Get the baseline hazard
baseline_hazard = cox.baseline_hazard_

# Calculate the predicted hazards for different values of PCA_vol1
import numpy as np

# Create a range of PCA_vol1 values (e.g., -2 standard deviations to +2 standard deviations)
pca_vol1_range = np.linspace(df['PCA_vol1'].min(), df['PCA_vol1'].max(), 100)
predicted_hazards = []

for value in pca_vol1_range:
    # Create a copy of the covariates with the given value of PCA_vol1
    covariates = pd.DataFrame({'PCA_vol1': [value] * len(baseline_hazard)})
    # Calculate the hazard for each time point
    hazard = np.exp(cox.predict_partial_hazard(covariates))
    predicted_hazards.append(hazard)

# Plot
plt.plot(baseline_hazard.index, np.array(predicted_hazards).flatten(), label="PCA_vol1 effect")
plt.title('Predicted Hazard Over Time for Different PCA_vol1 Values')
plt.xlabel('Time')
plt.ylabel('Hazard')
plt.legend()
plt.show()


In [ ]:
# Plot cumulative hazard
from lifelines import CoxPHFitter

# Fit the model
cox.fit(df, duration_col='duration', event_col='event')

# Plot cumulative hazard for PCA_vol1 ranges (e.g., 1 standard deviation below, mean, and 1 standard deviation above)
mean_vol1 = df['PCA_vol1'].mean()
std_vol1 = df['PCA_vol1'].std()

for value in [mean_vol1 - std_vol1, mean_vol1, mean_vol1 + std_vol1]:
    covariates = pd.DataFrame({'PCA_vol1': [value] * len(baseline_hazard)})
    cumulative_hazard = cox.predict_cumulative_hazard(covariates)
    plt.plot(cumulative_hazard.index, cumulative_hazard.values.flatten(), label=f"PCA_vol1 = {value:.2f}")

plt.title('Cumulative Hazard Over Time for Different PCA_vol1 Values')
plt.xlabel('Time')
plt.ylabel('Cumulative Hazard')
plt.legend()
plt.show()


In [ ]:
len(pca_vol1)

In [ ]:
data_vol1[1].shape

In [ ]:
np.shape(data_vol1)[0]

In [ ]:
data_vol1_flat.shape

In [ ]:
pd.DataFrame(pca_features)

In [ ]:
# PCA plus 
# average values
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom
import pandas as pd  # For handling features

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_SmallDependenceLowGrayLevelEmphasis.nrrd"
import re 
r1=re.split(string=nrrd_path, pattern='\/')
r2=re.split(string=r1[-1], pattern='_')
fname=r2[-1]
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim != 3:
    raise ValueError("Image array must be a 3D array")

num_slices = image_array.shape[0]

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)

# Define SOM parameters
som_width = 2
som_height = 2
som_depth = 1
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Prepare a list to hold pixel values from all slices
all_pixels = []

# Normalize and gather all pixel values from all slices
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    pixels = image_slice_norm.reshape(-1, input_len)
    all_pixels.append(pixels)

# Concatenate all pixel values into a single array
all_pixels = np.vstack(all_pixels)

# Train the SOM on the combined pixel values
som.random_weights_init(all_pixels)
som.train(all_pixels, 5000)

# Initialize a dictionary to hold pixel values grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height * som_depth)}

# Map each pixel in each slice to the trained SOM
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)

    # Map each pixel to the SOM
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_height * som_depth + w[1] * som_depth + (i % som_depth)

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]
            pixel_clusters[cluster_label].append(pixel_value)

# Calculate the average pixel value for each cluster
average_values = []
for label in range(som_width * som_height * som_depth):
    if pixel_clusters[label]:  # Ensure the cluster is not empty
        avg_value = np.mean(pixel_clusters[label])
        
        average_values.append(avg_value)
    else:
        average_values.append(np.nan)  # Handle empty clusters

# Convert average values to a DataFrame for further analysis
features_df = pd.DataFrame({
    'Cluster': range(som_width * som_height * som_depth),
    'Average_Value': average_values
})

print(features_df)

# You can now use `features_df` for Cox regression or further analysis.


In [ ]:
# average values
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom
import pandas as pd  # For handling features

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_SmallDependenceLowGrayLevelEmphasis.nrrd"
import re 
r1=re.split(string=nrrd_path, pattern='\/')
r2=re.split(string=r1[-1], pattern='_')
fname=r2[-1]
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim != 3:
    raise ValueError("Image array must be a 3D array")

num_slices = image_array.shape[0]

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)

# Define SOM parameters
som_width = 2
som_height = 2
som_depth = 1
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Prepare a list to hold pixel values from all slices
all_pixels = []

# Normalize and gather all pixel values from all slices
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    pixels = image_slice_norm.reshape(-1, input_len)
    all_pixels.append(pixels)

# Concatenate all pixel values into a single array
all_pixels = np.vstack(all_pixels)

# Train the SOM on the combined pixel values
som.random_weights_init(all_pixels)
som.train(all_pixels, 5000)

# Initialize a dictionary to hold pixel values grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height * som_depth)}

# Map each pixel in each slice to the trained SOM
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)

    # Map each pixel to the SOM
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_height * som_depth + w[1] * som_depth + (i % som_depth)

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]
            pixel_clusters[cluster_label].append(pixel_value)

# Calculate the average pixel value for each cluster
average_values = []
for label in range(som_width * som_height * som_depth):
    if pixel_clusters[label]:  # Ensure the cluster is not empty
        avg_value = np.mean(pixel_clusters[label])
        
        average_values.append(avg_value)
    else:
        average_values.append(np.nan)  # Handle empty clusters

# Convert average values to a DataFrame for further analysis
features_df = pd.DataFrame({
    'Cluster': range(som_width * som_height * som_depth),
    'Average_Value': average_values
})

print(features_df)

# You can now use `features_df` for Cox regression or further analysis.


In [ ]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
from minisom import MiniSom
from sklearn.decomposition import PCA

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_HighGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim != 3:
    raise ValueError("Image array must be a 3D array")

num_slices = image_array.shape[0]

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)

# Define SOM parameters
som_width = 2
som_height = 2
som_depth = 1
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Initialize a dictionary to hold pixel values grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height * som_depth)}

# Map each pixel in each slice to the trained SOM
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)

    # Map each pixel to the SOM
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_height * som_depth + w[1] * som_depth + (i % som_depth)

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]
            pixel_clusters[cluster_label].append(pixel_value)

# Calculate sum, product, and PCA for each cluster
features = []
for label in range(som_width * som_height * som_depth):
    if pixel_clusters[label]:  # Ensure the cluster is not empty
        pixel_values = np.array(pixel_clusters[label])
        
        # Calculate sum
        cluster_sum = np.sum(pixel_values)
        
        # Calculate product
        cluster_product = np.prod(pixel_values) if len(pixel_values) > 0 else 0
        
        # Calculate PCA and get the first principal component
        pca = PCA(n_components=1)
        pixel_values_reshaped = pixel_values.reshape(-1, 1)  # Reshape for PCA
        pca.fit(pixel_values_reshaped)
        pc1 = pca.transform(pixel_values_reshaped)

        # Handle the case where pc1 might be a single scalar
        first_pc_value = pc1[0, 0] if pc1.shape[0] > 0 else np.nan
        
        features.append({
            'Cluster': label,
            'Sum': cluster_sum,
            'Product': cluster_product,
            'PC1': first_pc_value
        })
    else:
        features.append({
            'Cluster': label,
            'Sum': np.nan,
            'Product': np.nan,
            'PC1': np.nan
        })

# Convert features to a DataFrame for further analysis
features_df = pd.DataFrame(features)
print(features_df)

# You can now use `features_df` for Cox regression or further analysis.


In [ ]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
from minisom import MiniSom
from sklearn.decomposition import PCA

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_HighGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim != 3:
    raise ValueError("Image array must be a 3D array")

num_slices = image_array.shape[0]

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)

# Define SOM parameters
som_width = 2
som_height = 2
som_depth = 1
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Initialize a dictionary to hold pixel values grouped by cluster labels
pixel_clusters = {label: [] for label in range(som_width * som_height * som_depth)}

# Map each pixel in each slice to the trained SOM, excluding the first and last slices
for i in range(1, num_slices - 1):  # Start from 1 and end at num_slices - 1
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)

    # Map each pixel to the SOM
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_height * som_depth + w[1] * som_depth + (i % som_depth)

    # Extract pixel values into lists grouped by cluster label
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            cluster_label = labels[x, y]
            pixel_value = image_slice[x, y]
            pixel_clusters[cluster_label].append(pixel_value)

# Calculate sum, product, and PCA for each cluster
features = []
for label in range(som_width * som_height * som_depth):
    if pixel_clusters[label]:  # Ensure the cluster is not empty
        pixel_values = np.array(pixel_clusters[label])
        print("pixel_values:",pixel_values[:])
        # Calculate sum
        cluster_sum = np.sum(pixel_values)
        
        # Calculate product
        cluster_product = np.prod(pixel_values) if len(pixel_values) > 0 else 0
        
        # Calculate PCA and get the first principal component
        pca = PCA(n_components=1)
        pixel_values_reshaped = pixel_values.reshape(-1, 1)  # Reshape for PCA
        pca.fit(pixel_values_reshaped)
        pc1 = pca.transform(pixel_values_reshaped)
        print('pc1',pc1)
        # Handle the case where pc1 might be a single scalar
        first_pc_value = pc1[0, 0] if pc1.shape[0] > 0 else np.nan
        
        features.append({
            'Cluster': label,
            'Sum': cluster_sum,
            'Product': cluster_product,
            'PC1': first_pc_value
        })
    else:
        features.append({
            'Cluster': label,
            'Sum': np.nan,
            'Product': np.nan,
            'PC1': np.nan
        })

# Convert features to a DataFrame for further analysis
features_df = pd.DataFrame(features)
print(features_df)

# You can now use `features_df` for Cox regression or further analysis.


In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from minisom import MiniSom

# Load the NRRD image using SimpleITK
nrrd_path = "C://Users//Administrator//Desktop//GB_ANALYSIS//GB_ANALYSIS_new//20_mm_final//TMH//gldm/105131012/105131012_original_gldm_HighGrayLevelEmphasis.nrrd"
image = sitk.ReadImage(nrrd_path)
image_array = sitk.GetArrayFromImage(image)

# Check image dimensions
if image_array.ndim not in (3, 4):
    raise ValueError("Image array must be a 3D or 4D array")

num_slices = image_array.shape[0] if image_array.ndim == 3 else 1

def normalize_slice(slice):
    max_val = np.max(slice)
    if max_val > 0:
        return (slice.astype(np.float32) / max_val).clip(0, 1)
    else:
        return np.zeros_like(slice, dtype=np.float32)

# Define SOM parameters (increased size)
som_width = 4  # Increased width
som_height =3  # Increased height
input_len = image_array.shape[-1] if image_array.ndim == 4 else 1
som = MiniSom(x=som_width, y=som_height, input_len=input_len, sigma=1.0, learning_rate=0.5)

# Collect all pixels from all slices for training
all_pixels = []

for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    pixels = image_slice_norm.reshape(-1, input_len)
    all_pixels.append(pixels)

# Concatenate all pixel arrays into one
all_pixels = np.vstack(all_pixels)

# Train the SOM on all pixels
som.random_weights_init(all_pixels)
som.train(all_pixels, 2000)  # Increased iterations

# Create a 3D array to hold cluster labels
labels_3d = np.zeros(image_array.shape, dtype=int)

# Map each voxel to the SOM and assign a label
for i in range(num_slices):
    image_slice = image_array[i]
    image_slice_norm = normalize_slice(image_slice)
    pixels = image_slice_norm.reshape(-1, input_len)
    
    labels = np.zeros((image_slice.shape[0], image_slice.shape[1]), dtype=int)
    for x in range(image_slice.shape[0]):
        for y in range(image_slice.shape[1]):
            w = som.winner(image_slice_norm[x, y])
            labels[x, y] = w[0] * som_width + w[1]

    labels_3d[i] = labels

    # Visualize the clusters as images
    cluster_images = []
    for label in range(som_width * som_height):
        cluster_image = np.zeros_like(image_slice, dtype=np.float32)
        cluster_image[labels == label] = image_slice[labels == label]
        cluster_images.append(cluster_image)

    # Plot the original and cluster images
    fig, axes = plt.subplots(1, som_width * som_height + 1, figsize=(15, 5))

    # Plot the original image
    axes[0].imshow(image_slice_norm, cmap='gray')
    axes[0].set_title(f'Original Image (Slice {i})')
    axes[0].axis('off')

    # Plot each cluster image
    for j, cluster_image in enumerate(cluster_images):
        axes[j + 1].imshow(cluster_image, cmap='gray')
        axes[j + 1].set_title(f'Cluster {j}')
        axes[j + 1].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
'C:/Users/Administrator/Desktop/gb_voxel_radiomics_analysis/example_images/gb_test.jpg'

### Auto encoder

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tensorflow import keras
from tensorflow.keras import layers
import cv2

# Load and preprocess the image
image_path = 'C:/Users/Administrator/Desktop/gb_voxel_radiomics_analysis/example_images/gb_test.jpg'  # Replace with your image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (128, 128))  # Resize for faster processing
image_normalized = image / 255.0  # Normalize to [0, 1]

# Prepare the data for the autoencoder
# Create patches by reshaping the image
image_patches = image_normalized.reshape(-1, 128 * 128)  # Shape: (num_pixels, 16384)

# Build the autoencoder
input_dim = image_patches.shape[1]  # 16384
encoding_dim = 64  # Dimension of the encoded representation

# Define the autoencoder model
autoencoder = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(encoding_dim, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(input_dim, activation='sigmoid')
])

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(image_patches, image_patches, epochs=100, batch_size=32, shuffle=True)  # Batch size adjusted

# Extract the encoder part
encoder = keras.Model(inputs=autoencoder.input, outputs=autoencoder.layers[2].output)

# Get the encoded representations
encoded_data = encoder.predict(image_patches)

# Plot the encoded data
plt.figure(figsize=(10, 6))
# Flatten the encoded data for plotting
encoded_flat = encoded_data.flatten()

# Create x values as pixel indices
x_values = np.arange(len(encoded_flat))

plt.scatter(x_values, encoded_flat, s=5, c='blue', alpha=0.5)
plt.title('Encoded Data from Autoencoder')
plt.xlabel('Pixel Index')
plt.ylabel('Encoded Value')
plt.grid()
plt.show()

# Apply K-means clustering
num_clusters = 4  # You can adjust this
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(encoded_data)

# Get cluster labels
cluster_labels = kmeans.labels_

# Reshape labels to match the original image shape
labels_image = cluster_labels.reshape(128, 128)

# Visualize the original image and the clustered segments
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image
axes[0].imshow(image_normalized, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Plot the clustered image
axes[1].imshow(labels_image, cmap='jet', alpha=0.5)
axes[1].set_title('Clustered Segmentation')
axes[1].axis('off')

# Optional: Show the clusters in different colors
segmented_image = np.zeros_like(image_normalized)
for cluster in range(num_clusters):
    segmented_image[labels_image == cluster] = cluster / num_clusters

axes[2].imshow(segmented_image, cmap   'jet')
axes[2].set_title('Segmented Image')
axes[2].axis('off')

plt.tight_layout()



plt.show()


In [ ]:
from lifelines import CoxPHFitter
import pandas as pd

# Create a DataFrame for your features
data = pd.DataFrame({
    'feature_1': cluster1_values,
    'feature_2': cluster2_values,
    # Add other features as needed
    'duration': durations,  # Time-to-event or duration
    'event': events         # Event occurred (1) or cen7jmjjjjjjjjjjjj9jsored (0)
})

# Fit the Cox model
cph = CoxPHFitter()
cph.fit(data, duration_col='duration', event_col='event')


In [ ]:
import pandas as pd

In [ ]:
df =pd.read_csv('C:/Users/Administrator/Desktop/GB_ANALYSIS/GB_ANALYSIS_new/gb_voxel_radiomics_survival_final.csv')

In [ ]:
df

In [ ]:
final_pca_df

In [ ]:
final_pca_df['death_disease_specific']=df['death_disease_specific']

In [ ]:
final_pca_df['death_interval_days']=df['death_interval_days']

In [ ]:
final_pca_df

In [ ]:
final_pca_df['death_disease_specific'].value_counts()

In [ ]:
df_tmh_pca=final_pca_df[['cluster1','cluster2','cluster3','cluster4','death_disease_specific','death_interval_days']]

In [ ]:
df_tmh_pca

In [ ]:
from lifelines import CoxPHFitter
import pandas as pd

# Fit the Cox model
cph = CoxPHFitter()
cph.fit(df_tmh_pca, duration_col='death_interval_days', event_col='death_disease_specific')


In [ ]:
# Display the summary of the fitted model
cph.summary

In [ ]:
cluster2 is statistically significant(p-value < 0.05).
cluster1 and cluster3 are not significant (p-value > 0.05).
cluster4 is marginally significant, with a small negative effect, but the p-value (0.052570) is just above the threshold of 0.05.

In [ ]:
print("C-index:", c_index)

In [ ]:
cph.plot()




In [ ]:
cph.plot_covariate_groups(, values=[value1, value2], cmap='coolwarm')


In [ ]:

cph.check_assumptions(df_tmh_pca, p_value_threshold=0.05)


In [ ]:
# Plot the survival function for specific values of the covariates
cph.plot()


In [ ]:
# Create a new DataFrame with the values for which you want to estimate survival
# Example: using the mean or median of covariates
mean_covariates = df_tmh.mean().to_frame().T  # Replace with actual covariate values as needed

# Estimate the survival function
survival_function = cph.predict_survival_function(mean_covariates)

# Plot the survival function
survival_function.plot()


In [ ]:
# Get coefficients and p-values
coefficients = cph.params_


print("Coefficients:\n", coefficients)
